In [ ]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_columns', None)

In [ ]:
tsjb_data = pd.read_table('data2/tsjbxx.txt',sep = ';')  #
tsjyygh_data = pd.read_table('data2/tsjyygh.txt',sep = ';')  #

file_list = [tsjb_data, tsjyygh_data]

In [ ]:
#读取两个字典
with open('book_id_count_map.json') as json_file:
    student_id_count_map = json.load(json_file)
    
with open('student_id_count_map.json') as json_file:
    student_id_count_map = json.load(json_file)

In [ ]:
#删除空属性
def drop_nan(file_list):
    for file_index in range(len(file_list)):
        for column in file_list[file_index].columns:
            if file_list[file_index][column].count() == 0:
                print(file_index, '.', column, 'is nan')
                file_list[file_index].drop(column, axis = 1, inplace=True)

In [ ]:
#构建maps映射表和get_value函数
data = pd.read_excel("data2/表结构- 竞赛-20190508.xlsx", sheet_name = None)
maps = {}
maps_list = []
for sheet in data:
    maps_list.append(sheet)

for i in range(7,9):
    for key, value in zip(data[maps_list[i]].iloc[:,0], data[maps_list[i]].iloc[:,2]):
        key = key.replace(' ','')
        maps[key] = value
        
#定义获取value函数
def get_value(key):
    if key.lower() not in maps:
        print('get_value收到了一个不存在的key值', key)
        return None
    return maps[key.lower()]

In [ ]:
#替换中文属性名
for file_index in range(len(file_list)):
    for i in range(len(file_list[file_index].columns)):
        if get_value(file_list[file_index].columns[i]) == None:
            print(file_index, '.', file_list[file_index].columns[i], ' is not found')
            continue;
        file_list[file_index].rename(columns={file_list[file_index].columns[i]:get_value(file_list[file_index].columns[i])}, inplace = True)

In [ ]:
#定义根据id获取书名的函数
def get_book_name(book_id, verbose = False):
    if len(tsjyygh_data[tsjyygh_data['图书条形码号'].isin([book_id])]) == 0:
        print("无法识别的图书条形码号")
        return None
    book_name = tsjyygh_data[tsjyygh_data['图书条形码号'].isin([book_id])].iloc[0, 3]
    if verbose:
        print(book_id, ": ", book_name)
    return book_name

In [ ]:
#借了某本书的所有学生列表
def get_student_list(book_id, verbose = False):
    if len(tsjyygh_data[tsjyygh_data['图书条形码号'].isin([book_id])]) == 0:
        print("该书无借阅记录")
        return None
    student_list = tsjyygh_data[tsjyygh_data['图书条形码号'].isin([book_id])]
    if verbose:
        print(student_list)
    return student_list['读者证件号'].values

In [ ]:
#某个学号的学生借过的所有书列表
def get_book_list(student_id, verbose = False):
    if len(tsjyygh_data[tsjyygh_data['读者证件号'].isin([student_id])]) == 0:
        print("该学号无借书记录")
        return None
    book_list = tsjyygh_data[tsjyygh_data['读者证件号'].isin([student_id])]
    if verbose:
        print(book_list)
    return book_list['书名'].values

In [ ]:
#构建用书id查找这本书被多少学生借了的字典     书号中有一个空  tsjyygh_data.iloc[494702,:]
i = 1
book_id_count_map = {}
temp = list(tsjyygh_data['图书条形码号'])

for book_id in tsjyygh_data['图书条形码号'][494700:]: 
    if book_id == None:
        continue
    book_id = str(book_id).strip()
    if book_id in book_id_count_map:
        book_id_count_map[book_id] = book_id_count_map[book_id] + 1
    else:
        book_id_count_map[book_id] = 1
    print('book ', i, '/', len(tsjyygh_data['图书条形码号']), ' : ', book_id, " : ", book_id_count_map[book_id])
    i = i + 1
    
#构建用学生id查找该学生借过多少次书的字典     学号是个int值
i = 1
student_id_count_map = {}
for student_id in tsjyygh_data['读者证件号']:
    if student_id == None:
        continue
    student_id = student_id.strip()
    if student_id in student_id_count_map:
        student_id_count_map[student_id] = student_id_count_map[student_id] + 1
    else:
        student_id_count_map[student_id] = 1
    print('student', i, '/', len(tsjyygh_data['读者证件号']), ' : ', student_id, " : ", student_id_count_map[student_id])
    i = i + 1

#保存两个字典
with open('book_id_count_map.json', 'w') as json_file:
    json_file.write(json.dumps(book_id_count_map))

with open('student_id_count_map.json', 'w') as json_file:
    json_file.write(json.dumps(student_id_count_map))

In [ ]:
list(tsjyygh_data['读者证件号']).count('201634027')

In [ ]:
tsjyygh_data[tsjyygh_data['读者证件号'].isin(['201634027'])]

In [23]:
book_id_count_map['nan']

1

In [26]:
abc = 88278+494700
tsjyygh_data.iloc[494702,:]

读者证件号      201409068
图书条形码号           NaN
定价               NaN
书名               NaN
借书日期      2014-10-29
应还日期             NaN
续借次数               0
归还日期      2014-12-01
Name: 494702, dtype: object